## Create model

In [1]:
# set project root to correct path
import sys
import os

proj_path = '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode'
if proj_path not in sys.path:
  sys.path.append(proj_path)

from set_path import set_path
set_path()

print(sys.path)
print(os.getcwd())

setting cwd to '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation'
['/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode/jupyter', '/home/payam/miniconda3/envs/tf2-gpu/lib/python39.zip', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/lib-dynload', '', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/site-packages', '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode']
/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation


In [3]:
import tensorflow as tf
print('GPU:', tf.config.list_physical_devices('GPU'))

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-09 11:40:30.687019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-09 11:40:30.757478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-09 11:40:30.757668: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


### zip

In [32]:
from tensorflow import keras
from tensorflow.keras import layers
from loader.vit_loader import load_model
# from tensorflow.python.keras.engine import keras_tensor
# keras_tensor.disable_keras_tensors()

def create_vit(input_shape, num_classes, model_path):
    base_vit = load_model(model_path)
    base_vit.build((None,*input_shape))
    # base_vit.layers.pop()
    # base_vit.trainable = False
    # base_vit.add(layers.Dense(num_classes, activation='sigmoid', name='my_output'))
    # print('------- base_vit -------')
    # print(base_vit.summary())
    ###################################################


    inputs = keras.Input(input_shape, name='my_input')
    base_output = base_vit(inputs)
    
    base_sliced_input = base_vit.input
    print('____inbound nodes:', base_vit.layers[-2].inbound_nodes)
    base_sliced_output = base_vit.layers[-2].output
    # x = layers.GlobalAveragePooling2D(name='my_avg_pool')(base_outputs)
    # x = layers.Dense(256, activation='relu', name='my_fc_1')(x)
    # x = layers.Dense(128, activation='relu', name='my_fc_2')(x)
    # x = layers.Dense(num_classes, activation='sigmoid', name='my_output')(x)
    model = keras.Model(inputs=base_sliced_input, outputs=base_sliced_output, name='my_vit')
    
    print('------- sliced model -------')
    print(model.summary())
    
    return model

AttributeError: module 'tensorflow.python.keras.engine.keras_tensor' has no attribute 'disable_keras_tensors'

In [33]:
path = 'base-models/ViT/vit_b16_patch16_224-i1k_pretrained.zip'
model = create_vit((224,224,3), 9, path)

# print(model.summary())
# for layer in model.layers:
#     print('layer name:', layer.name)

____inbound nodes: []


AttributeError: Layer layer_normalization_24 has no inbound nodes.

In [5]:



base_vit = load_model('base-models/ViT/vit_b16_patch16_224-i1k_pretrained.zip')

print("dir:", dir(base_vit)) # outputs all available functions/props to call
print(base_vit.summary())
for layer in base_vit.layers:
    print('layer name:', layer.name)



dir: ['_SCALAR_UPRANKING_ON', '_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activity_regularizer', '_add_trackable', '_add_trackable_child', '_add_variable_with_custom_getter', '_assert_compile_was_called', '_assert_weights_created', '_auto_config', '_auto_get_config', '_auto_track_sub_layers', '_autocast', '_autographed_call', '_base_model_initialized', '_build_input_shape', '_call_spec', '_callable_losses', '_captured_weight_regularizer', '_cast_single_input', '_check_call_args', '_check_sample_weight_warning', '_checkpoint', '_checkpoint_dependencies', '_clear_losses', '_cluster_coordi

### tf hub

In [12]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

input_shape = (224,224,3)
inputs = keras.Input(input_shape, name='my_input')

loaded_vit = hub.KerasLayer(
  "https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-b16-classification/1",
  trainable=False,
  # output_shape=[1280],
  # input_shape=[224,224,3]
)
base_output = loaded_vit(inputs)
# loaded_vit.build((None,224,224,3))

# for i, v in enumerate(loaded_vit.variables):
#     print('[{:03d}] {} [{}]'.format(i, v.name, v.shape))


model = keras.Model(inputs=inputs, outputs=base_output, name='my_vit')
print(model.summary())

Model: "my_vit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer_11 (KerasLayer)  (None, 1000)             86567656  
                                                                 
Total params: 86,567,656
Trainable params: 0
Non-trainable params: 86,567,656
_________________________________________________________________
None


In [17]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

input_shape = (224,224,3)
inputs = keras.Input(input_shape, name='my_input')

loaded_vit = hub.KerasLayer(
  "https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-b16-fe/1",
  trainable=False,
  # output_shape=[1280],
  # input_shape=[224,224,3]
)
loaded_vit.trainable = True
base_output = loaded_vit(inputs)
# x = layers.GlobalAveragePooling2D(name='my_avg_pool')(base_output)
# x = layers.Flatten(name='my_flatten')(base_resnet.output)
x = layers.Dense(256, activation='relu', name='my_fc_1')(base_output)
x = layers.Dense(128, activation='relu', name='my_fc_2')(x)
x = layers.Dense(10, activation='sigmoid', name='my_output')(x)
# loaded_vit.build((None,224,224,3))

# for i, v in enumerate(loaded_vit.variables):
#     print('[{:03d}] {} [{}]'.format(i, v.name, v.shape))


model = keras.Model(inputs=inputs, outputs=x, name='my_vit')
print(model.summary())

Model: "my_vit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer_16 (KerasLayer)  (None, 768)              85798656  
                                                                 
 my_fc_1 (Dense)             (None, 256)               196864    
                                                                 
 my_fc_2 (Dense)             (None, 128)               32896     
                                                                 
 my_output (Dense)           (None, 10)                1290      
                                                                 
Total params: 86,029,706
Trainable params: 86,029,706
Non-trainable params: 0
_________________________________________________________________
None


### hugging face

In [7]:
import tensorflow as tf
from transformers import TFViTModel

base_model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
base_model.trainable = False


# inputs
pixel_values = tf.keras.layers.Input(shape=(3,224,224), name='pixel_values', dtype='float32')

# model layer
vit = base_model.vit(pixel_values)[0]

classifier = tf.keras.layers.Dense(10, activation='softmax', name='outputs')(vit[:, 0, :])

# model
keras_model = tf.keras.Model(inputs=pixel_values, outputs=classifier)
keras_model.summary()

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pixel_values (InputLayer)   [(None, 3, 224, 224)]     0         
                                                                 
 vit (TFViTMainLayer)        TFBaseModelOutputWithPoo  86389248  
                             ling(last_hidden_state=(            
                             None, 197, 768),                    
                              pooler_output=(None, 76            
                             8),                                 
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 tf.__operators__.getitem_4   (None, 768)              0         
 (SlicingOpLambda)                                               
                                                           

## Test model

In [1]:
!python ../neural_nets/vit.py \
  --ouput_name=vit-test \
  --learning_rate=1e-4 --image_size=224 --epochs=2 --batch_size=16 --train_size=256 \
  --mode=train_then_eval --min_unfreeze_blocks=1 --max_unfreeze_blocks=1

setting cwd to '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation'
['/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode/neural_nets', '/home/payam/miniconda3/envs/tf2-gpu/lib/python39.zip', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/lib-dynload', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/site-packages', '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode']
/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation
2024-04-09 18:16:14.706485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 18:16:15.304822: W tensorflow/compiler/xla/stream_executor/platform/default/dso_

## Mini-batch Test

In [17]:
!python ../neural_nets/vit_test.py \
  --ouput_name=vit-miniBatchTest \
  --image_size=224 --epochs=100 --batch_size=8 --train_size=8 \
  --mode=train_then_eval --min_unfreeze_blocks=0 --max_unfreeze_blocks=0

setting cwd to '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation'
['/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode/neural_nets', '/home/payam/miniconda3/envs/tf2-gpu/lib/python39.zip', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/lib-dynload', '/home/payam/miniconda3/envs/tf2-gpu/lib/python3.9/site-packages', '/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation/mycode']
/mnt/samba/research/shield/projects/payamfz/medical-ssl-segmentation
2024-04-09 17:13:23.812909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 17:13:24.354792: W tensorflow/compiler/xla/stream_executor/platform/default/dso_